In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
#Cargo el dataset y ajusto las variables que interesan
df = pd.read_csv("D:/Downloads/speeddating.csv")
df['samerace'] = df['samerace'].str.replace('[^0-9]', '', regex=True).astype('int64')
df['decision'] = df['decision'].str.replace('[^0-9]', '', regex=True).astype('int64')
df['decision_o'] = df['decision_o'].str.replace('[^0-9]', '', regex=True).astype('int64')

In [4]:
#Veo como es el dataset
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,b'',1.0,b'female',21.0,27.0,6.0,b'[4-6]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',0,2.0,4.0,b'[2-5]',b'[2-5]',b'Law',35.0,20.0,20.0,20.0,0.0,5.0,b'[21-100]',b'[16-20]',b'[16-20]',b'[16-20]',b'[0-15]',b'[0-15]',6.0,8.0,8.0,8.0,8.0,6.0,b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',15.0,20.0,20.0,15.0,15.0,15.0,b'[0-15]',b'[16-20]',b'[16-20]',b'[0-15]',b'[0-15]',b'[0-15]',6.0,8.0,8.0,8.0,7.0,b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',6.0,9.0,7.0,7.0,6.0,5.0,b'[6-8]',b'[9-10]',b'[6-8]',b'[6-8]',b'[6-8]',b'[0-5]',9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,b'[9-10]',b'[0-5]',b'[6-8]',b'[9-10]',b'[0-5]',b'[0-5]',b'[0-5]',b'[0-5]',b'[0-5]',b'[6-8]',b'[9-10]',b'[0-5]',b'[9-10]',b'[9-10]',b'[9-10]',b'[6-8]',b'[0-5]',0.14,b'[0-0.33]',3.0,2.0,4.0,b'[0-4]',b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,1,0,b'0'
1,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',0,2.0,4.0,b'[2-5]',b'[2-5]',b'Law',60.0,0.0,0.0,40.0,0.0,0.0,b'[21-100]',b'[0-15]',b'[0-15]',b'[21-100]',b'[0-15]',b'[0-15]',7.0,8.0,10.0,7.0,7.0,5.0,b'[6-8]',b'[6-8]',b'[9-10]',b'[6-8]',b'[6-8]',b'[0-5]',15.0,20.0,20.0,15.0,15.0,15.0,b'[0-15]',b'[16-20]',b'[16-20]',b'[0-15]',b'[0-15]',b'[0-15]',6.0,8.0,8.0,8.0,7.0,b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',7.0,8.0,7.0,8.0,5.0,6.0,b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[0-5]',b'[6-8]',9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,b'[9-10]',b'[0-5]',b'[6-8]',b'[9-10]',b'[0-5]',b'[0-5]',b'[0-5]',b'[0-5]',b'[0-5]',b'[6-8]',b'[9-10]',b'[0-5]',b'[9-10]',b'[9-10]',b'[9-10]',b'[6-8]',b'[0-5]',0.54,b'[0.33-1]',3.0,2.0,4.0,b'[0-4]',b'[0-3]',b'[3-5]',7.0,5.0,b'[6-8]',b'[5-6]',1.0,1,0,b'0'
2,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'Asian/Pacific Islander/Asian-American',1,2.0,4.0,b'[2-5]',b'[2-5]',b'Law',19.0,18.0,19.0,18.0,14.0,12.0,b'[16-20]',b'[16-20]',b'[16-20]',b'[16-20]',b'[0-15]',b'[0-15]',10.0,10.0,10.0,10.0,10.0,10.0,b'[9-10]',b'[9-10]',b'[9-10]',b'[9-10]',b'[9-10]',b'[9-10]',15.0,20.0,20.0,15.0,15.0,15.0,b'[0-15]',b'[16-20]',b'[16-20]',b'[0-15]',b'[0-15]',b'[0-15]',6.0,8.0,8.0,8.0,7.0,b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',b'[6-8]',5.0,8.0,9.0,

In [18]:
#Creo un nuevo dataset con las columnas que me interesan 
df2 = df[['gender', 'age', 'd_age', 'samerace', 'field', 'attractive_o', 'sinsere_o', 'intelligence_o', 'ambitous_o', 'shared_interests_o', 'funny_o', 'decision_o']].copy()

In [19]:
#Tiene valores vacios, hay que ver que hacer con ellos 
df2.isnull().sum()

gender                   0
age                     95
d_age                    0
samerace                 0
field                    0
attractive_o           212
sinsere_o              287
intelligence_o         306
ambitous_o             722
shared_interests_o    1076
funny_o                360
decision_o               0
dtype: int64

In [15]:
#Tiene valores maximos falsos, hay que cambiarlos 
df2['attractive_o'].max()

10.5

In [44]:
df2.loc[df2['attractive_o'] == 10.5] = 10
df2.loc[df2['funny_o'] == 11.0] = 10

In [70]:
# Encuentro dato falso, lo borro
df2.loc[df2["decision_o"]==10]

,gender,age,d_age,samerace,field,attractive_o,sinsere_o,intelligence_o,ambitous_o,shared_interests_o,funny_o,decision_o


In [69]:
df2 = df2.drop(6608)

In [73]:
#Defino un nuevo dataset sin los valores vacios 
df3 = df2.dropna(subset=['attractive_o', 'intelligence_o', 'funny_o','ambitous_o', 'shared_interests_o', 'sinsere_o'])

In [74]:
#Sigo teniendo bastantes valores, no me pongo mal por sacar los NA
df3.shape

(7030, 12)

In [75]:
#Entreno el modelo
log = LogisticRegression(penalty= None)
x= df3[["d_age",'samerace','attractive_o', 'intelligence_o', 'funny_o','ambitous_o', 'shared_interests_o', 'sinsere_o']].to_numpy()
y= df3["decision_o"].to_numpy()
log.fit(x,y)

LogisticRegression(penalty=None)

In [76]:
log.predict_proba(x)

array([[0.57754063, 0.42245937],
       [0.49796725, 0.50203275],
       [0.04447   , 0.95553   ],
       ...,
       [0.97813481, 0.02186519],
       [0.67643063, 0.32356937],
       [0.27131401, 0.72868599]])

In [104]:
#Sirve para ver los pesos de cada feature (la diferencia de edad resta, el atractivo suma)
log.coef_

array([[-0.01783868, -0.01947686,  0.5496946 ,  0.02439948,  0.264757  ,
        -0.16668203,  0.26889456, -0.10955181]])

In [105]:
log.intercept_

array([-5.22142736])

In [77]:
predictions = log.predict(x)

In [80]:
#Exactitud
score = log.score(x, y)
print(score)

0.7480796586059744
